In [36]:
from abc import ABC, abstractmethod
import pandas as pd
import numpy as np
from scipy.stats import norm
from functools import reduce
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from DateRanges import electionPeriodBoolsDF

class FinancialInstrument(ABC):
    """
    Abstract base class representing a generic financial instrument.
    """

    @property
    @abstractmethod
    def log_returns(self):
        """
        Abstract property representing the log returns as a pandas DataFrame.
        Must be implemented by subclasses.
        """
        pass
    @property
    @abstractmethod
    def instrument_type(self):
        """
        Abstract property representing the instrument type as a string.
        Must be implemented by subclasses.
        """
        pass

    @property
    @abstractmethod
    def ticker(self):
        """
        Abstract property representing the ticker as a string.
        Must be implemented by subclasses.
        """
        pass

    @property
    @abstractmethod
    def period(self):
        """
        Abstract property representing the type of period as a string.
        Must be implemented by subclasses.
        """
        pass


    def calculate_volatility(self, annualize=True):
        """
        This function calculates the daily or annualized volatility of log returns.

        Args:
            annualize (Boolean): If True, returns annualized volatility. Else, returns daily volatility.
        
        Returns:
            Series with volatility of each asset.
        """
        daily_vol = self.log_returns.std()
        if annualize:
            return daily_vol * np.sqrt(252)
        return daily_vol
    
    def calculate_variance(self, annualize=True):
        """
        This function calculates the daily or annualized variance of log returns.

        Args:
            annualize (Boolean): If True, returns annualized variance. Else, returns daily variance.
        
        Returns:
            Float with the variance of the log returns.
        """
        daily_variance = self.log_returns.var()
        if annualize:
            return float(daily_variance.iloc[0]) * 252
        return float(daily_variance.iloc[0])

    def correlation_matrix(self, others):
        """
        This function calculates the correlation matrix of the log returns.

        Args:
            otherList (list): List of other Financial Instruments.

        Returns:
            DataFrame with correlation matrix.
        """
        log_return_df_list = [self.log_returns]
        for other in others:
            log_return_df_list += [other.log_returns]
        merged_df = reduce(lambda x, y: pd.merge(x, y, how='inner', on="Date"), log_return_df_list)
        print(merged_df)
        return merged_df.corr()
    
    def covariance_matrix(self, others):
        """
        This function calculates the covariance matrix of the log returns.

        Args:
            otherList (list): List of other Financial Instruments.

        Returns:
            DataFrame with covariance matrix.
        """
        log_return_df_list = [self.log_returns]
        for other in others:
            log_return_df_list += [other.log_returns]
        merged_df = reduce(lambda x, y: pd.merge(x, y, how='inner', on="Date"), log_return_df_list)
        return merged_df.cov()

    def calculate_beta(self, benchmark):
        """
        This function calculates the beta between self and a benchmark Financial Instrument.

        Args:
            benchmark (FinancialInstrument): benchmark Financial Instrument.

        Returns:
            DataFrame with correlation matrix.
        """
        cov_matrix = self.covariance_matrix([benchmark])
        covariance = cov_matrix.iloc[0, 1]
        benchmark_var = benchmark.calculate_variance(annualize=False) # There was an issue with dividing by zero when working with ETFs, check on this again
        benchmark_var = cov_matrix.iloc[1,1]
        beta = float(covariance / benchmark_var)
        return beta
    
    def summary(self):
        vol_series = self.calculate_volatility()
        vol = float(vol_series.iloc[0])
        return (f"Type: {self.instrument_type}, "
                f"Ticker: {self.ticker}, "
                f"Period: {self.period}, "
                f"Volatility: {vol}")


class ETF(FinancialInstrument):
    """
    Class representing an Exchange-Traded Fund (ETF).
    """
    def __init__(self, ticker, period):
        """
        This is the constructor for the ETF class. Ticker is the string of the ticker and period is an integer
        that represents which time period is wanted to be used.
            

        Args:
            ticker (string): The ticker of the ETF.
            period (integer): The time period that should be considered.
                period = 1 -> election periods
                period = -1 -> non-election periods
                period = anything else -> total time period
        
        Returns:
            None
        """
        self.tickerCode = ticker
        self.df = pd.read_csv("../Data/ETFData/merged_cleaned_etf_data.csv")
        self.df['Date'] = pd.to_datetime(self.df['Date'])
        self.prices = self.df.set_index('Date')[[ticker + " Adj Close"]]
        self.periodCode = period
        merged_df = pd.merge(self.prices, electionPeriodBoolsDF, left_index=True, right_index=True, how='inner')
        if period == 1:
            merged_df = merged_df[merged_df['In an Election Period']]
        elif period == -1:
            merged_df = merged_df[~merged_df['In an Election Period']]
        merged_df = merged_df.drop(columns=['In an Election Period'])
        self.prices = merged_df

    
    @property
    def log_returns(self):
        return np.log(self.prices / self.prices.shift(1)).dropna().rename(columns={self.ticker + " Adj Close": self.ticker + " Log Return"})

    @property
    def instrument_type(self):
        return "ETF"
    
    @property
    def ticker(self):
        return self.tickerCode
    
    @property
    def period(self):
        if self.periodCode == 1:
            return "Election Periods"
        if self.periodCode == -1:
            return "Non-Election Periods"
        return "Total Time Period"


    def calculate_VaR(self, confidence_level=0.95):
        """
        This function calculates the Value at Risk (VaR) for log returns at a given confidence level.

        Args:
            confidence_level (float): The confidence level for the VaR calculation.
        
        Returns:
            Series with VaR of each asset.
        """
        mean = self.log_returns.mean()
        std_dev = self.log_returns.std()
        z_score = norm.ppf(1 - confidence_level)
        return -(mean + z_score * std_dev)
    
    def calculate_ES(self, confidence_level=0.95):
        """
        This function calculates the Expected Shortfall (ES) for the log returns at a given confidence level.

        Args:
            confidence_level (float): The confidence level for the ES calculation.
        
        Returns:
            Series with ES of each asset.
        """
        VaR = self.calculate_VaR(confidence_level)
        tail_losses = self.log_returns[self.log_returns < -VaR]
        return -tail_losses.mean()

class Future(FinancialInstrument):
    """
    Class representing a Future.
    """
    def __init__(self, ticker, period):
        """
        This is the constructor for the Future class. Ticker is the string of the ticker and period is an integer
        that represents which time period is wanted to be used.
            

        Args:
            ticker (string): The ticker of the Future.
            period (integer): The time period that should be considered.
                period = 1 -> election periods
                period = -1 -> non-election periods
                period = anything else -> total time period
        
        Returns:
            None
        """
        self.tickerCode = ticker
        self.df = pd.read_csv("../Data/FuturesData/merged_cleaned_futures_data.csv")
        self.df = self.df[self.df[ticker + " LAST"] >= 0]
        self.df['Date'] = pd.to_datetime(self.df['Date'])
        self.prices = self.df.set_index('Date')[[ticker + " LAST"]]
        self.periodCode = period
        merged_df = pd.merge(self.prices, electionPeriodBoolsDF, left_index=True, right_index=True, how='inner')
        if period == 1:
            merged_df = merged_df[merged_df['In an Election Period']]
        elif period == -1:
            merged_df = merged_df[~merged_df['In an Election Period']]
        merged_df = merged_df.drop(columns=['In an Election Period'])
        self.prices = merged_df
    
    @property
    def log_returns(self):
        return np.log(self.prices / self.prices.shift(1)).dropna().rename(columns={self.tickerCode + " LAST": self.tickerCode + " Log Return"})

    @property
    def instrument_type(self):
        return "Future"
    
    @property
    def ticker(self):
        return self.tickerCode

    @property
    def period(self):
        if self.periodCode == 1:
            return "Election Periods"
        if self.periodCode == -1:
            return "Non-Election Periods"
        return "Total Time Period"


    def calculate_VaR(self, confidence_level=0.95):
        """
        This function calculates the Value at Risk (VaR) for log returns at a given confidence level.

        Args:
            confidence_level (float): The confidence level for the VaR calculation.
        
        Returns:
            Series with VaR of each asset.
        """
        mean = self.log_returns.mean()
        std_dev = self.log_returns.std()
        z_score = norm.ppf(1 - confidence_level)
        return -(mean + z_score * std_dev)
    
    def calculate_ES(self, confidence_level=0.95):
        """
        This function calculates the Expected Shortfall (ES) for the log returns at a given confidence level.

        Args:
            confidence_level (float): The confidence level for the ES calculation.
        
        Returns:
            Series with ES of each asset.
        """
        VaR = self.calculate_VaR(confidence_level)
        tail_losses = self.log_returns[self.log_returns < -VaR]
        return -tail_losses.mean()